In [13]:
import requests
from requests.models import Response
from requests.structures import CaseInsensitiveDict


In [2]:
response: Response = requests.get('https://simple-books-api.glitch.me/status')
print(response)
print(response.status_code)
print(response.json())
print(response.headers)

<Response [200]>
200
{'status': 'OK'}
{'Date': 'Mon, 25 Dec 2023 03:39:25 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '15', 'Connection': 'keep-alive', 'x-powered-by': 'Express', 'etag': 'W/"f-v/Y1JusChTxrQUzPtNAKycooOTA"'}


In [14]:
import json
import requests

def get_all_books():
    response = requests.get('https://simple-books-api.glitch.me/books')
    res = response.json()
    books_str = json.dumps(res)
    return books_str
get_all_books()

'[{"id": 1, "name": "The Russian", "type": "fiction", "available": true}, {"id": 2, "name": "Just as I Am", "type": "non-fiction", "available": false}, {"id": 3, "name": "The Vanishing Half", "type": "fiction", "available": true}, {"id": 4, "name": "The Midnight Library", "type": "fiction", "available": true}, {"id": 5, "name": "Untamed", "type": "non-fiction", "available": true}, {"id": 6, "name": "Viscount Who Loved Me", "type": "fiction", "available": true}]'

In [12]:
# import json
# import requests

# def get_all_books():
#     response = requests.get('https://simple-books-api.glitch.me/books')
#     res = response.json()
#     return res

# books = get_all_books()
# books_str = json.dumps(books)

# get_all_books()
# books_str


'[{"id": 1, "name": "The Russian", "type": "fiction", "available": true}, {"id": 2, "name": "Just as I Am", "type": "non-fiction", "available": false}, {"id": 3, "name": "The Vanishing Half", "type": "fiction", "available": true}, {"id": 4, "name": "The Midnight Library", "type": "fiction", "available": true}, {"id": 5, "name": "Untamed", "type": "non-fiction", "available": true}, {"id": 6, "name": "Viscount Who Loved Me", "type": "fiction", "available": true}]'

## Get List of Books


In [9]:
import json
def get_all_books():
        response = requests.get('https://simple-books-api.glitch.me/books')
        res = response.json()
        return res
get_all_books()

[{'id': 1, 'name': 'The Russian', 'type': 'fiction', 'available': True},
 {'id': 2, 'name': 'Just as I Am', 'type': 'non-fiction', 'available': False},
 {'id': 3, 'name': 'The Vanishing Half', 'type': 'fiction', 'available': True},
 {'id': 4,
  'name': 'The Midnight Library',
  'type': 'fiction',
  'available': True},
 {'id': 5, 'name': 'Untamed', 'type': 'non-fiction', 'available': True},
 {'id': 6,
  'name': 'Viscount Who Loved Me',
  'type': 'fiction',
  'available': True}]

## Get a Single Book


In [23]:
response: Response = requests.get('https://simple-books-api.glitch.me/books/1')
status_code : int = response.status_code
print(status_code)
json_obj : dict = response.json()
pprint.pprint(json_obj)


200
{'author': 'James Patterson and James O. Born',
 'available': True,
 'current-stock': 12,
 'id': 1,
 'isbn': '1780899475',
 'name': 'The Russian',
 'price': 12.98,
 'type': 'fiction'}


## Get Book ID by Name


In [17]:
def get_book_id(name):
    response = requests.get('https://simple-books-api.glitch.me/books')
    res = response.json()

    for book in res:
        if book['name'] == name:
            return book['id']



## Get Price BY Name


In [19]:
def getprice(name):
    bookid = get_book_id(name)
    response = requests.get(f"https://simple-books-api.glitch.me/books/{bookid}")
    res = response.json()
    fres = res['price']
    new = json.dumps(fres)
    return new

getprice("The Russian")

'12.98'

## Get Single Book By Name


In [7]:
def get_single_book(name):
    bookid = get_book_id(name)
    response = requests.get(f"https://simple-books-api.glitch.me/books/{bookid}")
    res = response.json()
    return res

get_single_book("The Russian")

{'id': 1,
 'name': 'The Russian',
 'author': 'James Patterson and James O. Born',
 'isbn': '1780899475',
 'type': 'fiction',
 'price': 12.98,
 'current-stock': 12,
 'available': True}

In [20]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

In [25]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}] # is me user ka message aye ga
    tools = [

        {
            "type": "function",
            "function": {
                "name": "get_all_books",
                "description": "Get the list of books",
            },
        },
       
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls
    display("* First Reponse Tool Calls: ", list(tool_calls))

    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_all_books": get_all_books,
            }  # only one function in this example, but you can have multiple
        
        messages.append(response_message)  # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call()    
            messages.append({
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": function_name,
                                "content": function_response,
                            })
                          # extend conversation with function response
        display("* Second Request Messages: ", list(messages))
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content
        # return second_response.choices[0].message.content

In [26]:
run_conversation("which books do you have")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_ea30LbXG8vXw8ZGbP1DjK4B0', function=Function(arguments='{}', name='get_all_books'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_ea30LbXG8vXw8ZGbP1DjK4B0', function=Function(arguments='{}', name='get_all_books'), type='function')]

'* Second Request Messages: '

[{'role': 'user', 'content': 'which books do you have'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ea30LbXG8vXw8ZGbP1DjK4B0', function=Function(arguments='{}', name='get_all_books'), type='function')]),
 {'tool_call_id': 'call_ea30LbXG8vXw8ZGbP1DjK4B0',
  'role': 'tool',
  'name': 'get_all_books',
  'content': '[{"id": 1, "name": "The Russian", "type": "fiction", "available": true}, {"id": 2, "name": "Just as I Am", "type": "non-fiction", "available": false}, {"id": 3, "name": "The Vanishing Half", "type": "fiction", "available": true}, {"id": 4, "name": "The Midnight Library", "type": "fiction", "available": true}, {"id": 5, "name": "Untamed", "type": "non-fiction", "available": true}, {"id": 6, "name": "Viscount Who Loved Me", "type": "fiction", "available": true}]'}]

* Second Response:  {'id': 'chatcmpl-8a4flMLBtJK5wp59b4UiTuXn2G82R', 'choices': [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='I have the following books available:\n\n1. The Russian (fiction)\n2. The Vanishing Half (fiction)\n3. The Midnight Library (fiction)\n4. Untamed (non-fiction)\n5. Viscount Who Loved Me (fiction)', role='assistant', function_call=None, tool_calls=None), logprobs=None)], 'created': 1703608461, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_772e8125bb', 'usage': CompletionUsage(completion_tokens=49, prompt_tokens=187, total_tokens=236)}


'I have the following books available:\n\n1. The Russian (fiction)\n2. The Vanishing Half (fiction)\n3. The Midnight Library (fiction)\n4. Untamed (non-fiction)\n5. Viscount Who Loved Me (fiction)'

In [21]:
# from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# # from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

# def run_conversation(main_request: str)->str:
#     # Step 1: send the conversation and available functions to the model
#     messages = [{"role": "user", "content": main_request}] # is me user ka message aye ga
#     tools = [

#         {
#             "type": "function",
#             "function": {
#                 "name": "getprice",
#                 "description": "Get the price by its name",
#                 "parameters": {
#                     "type": "object",
#                     "properties": {
#                         "name": {
#                             "type": "string",
#                             "description": "name of book, e.g. The Russian",
#                         }},
#                     "required": ["name"],
#                 },
#             },
#         },
       
#     ]

#     # First Request
#     response: ChatCompletion = client.chat.completions.create(
#         model="gpt-3.5-turbo-1106",
#         messages=messages,
#         tools=tools,
#         tool_choice="auto",  # auto is default, but we'll be explicit
#     )
#     response_message: ChatCompletionMessage = response.choices[0].message
#     display("* First Response: ", dict(response_message))

#     tool_calls = response_message.tool_calls
#     display("* First Reponse Tool Calls: ", list(tool_calls))

#     # Step 2: check if the model wanted to call a function
#     if tool_calls:
#         # Step 3: call the function
#         # Note: the JSON response may not always be valid; be sure to handle errors
#         available_functions = {
#             "getprice": getprice,
#             }  # only one function in this example, but you can have multiple
        
#         messages.append(response_message)  # extend conversation with assistant's reply
        
#         # Step 4: send the info for each function call and function response to the model
#         for tool_call in tool_calls:
#             function_name = tool_call.function.name
#             function_to_call = available_functions[function_name]
#             function_args = json.loads(tool_call.function.arguments)
#             function_response = function_to_call(
#                 name=function_args.get(), # prompt se location ko put karo location me
#             )    
#             messages.append({
#                                 "tool_call_id": tool_call.id,
#                                 "role": "tool",
#                                 "name": function_name,
#                                 "content": function_response,
#                             })
#                           # extend conversation with function response
#         display("* Second Request Messages: ", list(messages))
#         second_response: ChatCompletion = client.chat.completions.create(
#             model="gpt-3.5-turbo-1106",
#             messages=messages,
#         )  # get a new response from the model where it can see the function response
#         print("* Second Response: ", dict(second_response))
#         return second_response.choices[0].message.content
#         # return second_response.choices[0].message.content

In [22]:
run_conversation("what is the price of The Russian book")


'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_pkd1KSKhTpYfVzD7SjeFXkbu', function=Function(arguments='{"name": "The Russian"}', name='getprice'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_pkd1KSKhTpYfVzD7SjeFXkbu', function=Function(arguments='{"name": "The Russian"}', name='getprice'), type='function')]

'* Second Request Messages: '

[{'role': 'user', 'content': 'what is the price of The Russian book'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_pkd1KSKhTpYfVzD7SjeFXkbu', function=Function(arguments='{"name": "The Russian"}', name='getprice'), type='function')]),
 {'tool_call_id': 'call_pkd1KSKhTpYfVzD7SjeFXkbu',
  'role': 'tool',
  'name': 'getprice',
  'content': '12.98'}]

* Second Response:  {'id': 'chatcmpl-8a3uERADZxDQD1aAep5BdTzOOSW5J', 'choices': [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The price of "The Russian" book is $12.98.', role='assistant', function_call=None, tool_calls=None), logprobs=None)], 'created': 1703605514, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_772e8125bb', 'usage': CompletionUsage(completion_tokens=14, prompt_tokens=41, total_tokens=55)}


'The price of "The Russian" book is $12.98.'

In [10]:
run_conversation("what is the price of The Russian book")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_UFiXgfHvfSXTFwnasoeiSrjL', function=Function(arguments='{"name":"The Russian"}', name='getprice'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_UFiXgfHvfSXTFwnasoeiSrjL', function=Function(arguments='{"name":"The Russian"}', name='getprice'), type='function')]

i am here


'* Second Request Messages: '

[{'role': 'assistant',
  'content': 'You are only a bookstore assistant. You should only tell users about the names, prices, availability, and other details of books. If any other question arises, you should refuse to answer it.'},
 {'role': 'user', 'content': 'what is the price of The Russian book'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_UFiXgfHvfSXTFwnasoeiSrjL', function=Function(arguments='{"name":"The Russian"}', name='getprice'), type='function')]),
 {'tool_call_id': 'call_UFiXgfHvfSXTFwnasoeiSrjL',
  'role': 'tool',
  'name': 'getprice',
  'content': 12.98}]

BadRequestError: Error code: 400 - {'error': {'message': "'$.messages[3].content' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [12]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

In [9]:
def getprice(name):
    bookid = get_book_id(name)
    response = requests.get(f"https://simple-books-api.glitch.me/books/{bookid}")
    res = response.json()
    return res['price']

getprice("The Russian")

12.98

In [2]:
import json
def get_current_weather(location:str, unit:str="fahrenheit")->str:
    """Get the current weather in a given location"""  # d
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

get_current_weather("Tokyo")

'{"location": "Tokyo", "temperature": "10", "unit": "celsius"}'

In [18]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}] # is me user ka message aye ga
    tools = [ # is me hum functions ki description baten ge
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106", 
        messages=messages,# user message
        tools=tools, # functions
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls # ye function aur parameter lae ga
    display("* First Reponse Tool Calls: ", list(tool_calls))

    # Step 2: check if the model wanted to call a function
    if tool_calls:  # agr tools call me kuch ho
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = { # available func me wo func mujud b hu
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        
        messages.append(response_message)  # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name # function ka naam batao
            function_to_call = available_functions[function_name] # check karo available function me wo function be he 
            function_args = json.loads(tool_call.function.arguments) # function ke arguments nikalo
            function_response = function_to_call(
                location=function_args.get("location"), # prompt se location ko put karo location me 
                unit=function_args.get("unit"),# prompt se unit ko put karo unit me 
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        display("* Second Request Messages: ", list(messages)) # messages me jo b he dkado
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content


In [19]:
run_conversation("What's the weather like in San Francisco, Tokyo, and Paris?")


'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_UNFPqlH2vdkbqMd0WAhvjZYe', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_JePnZe5BMHgGzS1XaIVGrccC', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_AW3kgM6JBSFMWSQRw6YqeELi', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_UNFPqlH2vdkbqMd0WAhvjZYe', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_JePnZe5BMHgGzS1XaIVGrccC', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_AW3kgM6JBSFMWSQRw6YqeELi', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]

'* Second Request Messages: '

[{'role': 'user',
  'content': "What's the weather like in San Francisco, Tokyo, and Paris?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_UNFPqlH2vdkbqMd0WAhvjZYe', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_JePnZe5BMHgGzS1XaIVGrccC', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_AW3kgM6JBSFMWSQRw6YqeELi', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_UNFPqlH2vdkbqMd0WAhvjZYe',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}'},
 {'tool_call_id': 'call_JePnZe5BMHgGzS1XaIVGrccC',
  'role': 

* Second Response:  {'id': 'chatcmpl-8ZWc89a9VPKIYkuxih4s9RApV9u5E', 'choices': [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The current weather in San Francisco is 72°F, in Tokyo it is 10°C, and in Paris it is 22°C.', role='assistant', function_call=None, tool_calls=None), logprobs=None)], 'created': 1703477540, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_772e8125bb', 'usage': CompletionUsage(completion_tokens=28, prompt_tokens=169, total_tokens=197)}


'The current weather in San Francisco is 72°F, in Tokyo it is 10°C, and in Paris it is 22°C.'

In [34]:
def welcome(name="john"):
    """Welcome user of a given name"""
    print(f"Welcome {name}")


In [37]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}] # is me user ka message aye ga
    tools = [ # is me hum functions ki description baten ge
        {
            "type": "function",
            "function": {
                "name": "welcome",
                "description": "Welcome user of a given name",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The name of user, e.g. John Doe , Ali",
                        },
                        
                    },
                    "required": ["name"],
                },
            },
        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106", 
        messages=messages,# user message
        tools=tools, # functions
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls # ye function aur parameter lae ga
    display("* First Reponse Tool Calls: ", list(tool_calls))

    # Step 2: check if the model wanted to call a function
    if tool_calls:  # agr tools call me kuch ho
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = { # available func me wo func mujud b hu
            "welcome": welcome,
        }  # only one function in this example, but you can have multiple
        
        messages.append(response_message)  # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name # function ka naam batao
            function_to_call = available_functions[function_name] # check karo available function me wo function be he 
            function_args = json.loads(tool_call.function.arguments) # function ke arguments nikalo
            function_response = function_to_call(
                name=function_args.get("name"), # prompt se location ko put karo location me 
                
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        display("* Second Request Messages: ", list(messages)) # messages me jo b he dkado
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content


In [38]:
run_conversation("my name is rafi")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_muad3ZkFViKkHK4pJnpBMFA6', function=Function(arguments='{"name":"rafi"}', name='welcome'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_muad3ZkFViKkHK4pJnpBMFA6', function=Function(arguments='{"name":"rafi"}', name='welcome'), type='function')]

Welcome rafi


'* Second Request Messages: '

[{'role': 'user', 'content': 'my name is rafi'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_muad3ZkFViKkHK4pJnpBMFA6', function=Function(arguments='{"name":"rafi"}', name='welcome'), type='function')]),
 {'tool_call_id': 'call_muad3ZkFViKkHK4pJnpBMFA6',
  'role': 'tool',
  'name': 'welcome',
  'content': None}]

BadRequestError: Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null.", 'type': 'invalid_request_error', 'param': 'messages.[2].content', 'code': None}}